In [44]:
import os

In [45]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflow'

In [46]:
os.chdir("C:/Users/pc/Desktop/GitHub repos/End-to-end-ML-project-with-MLflow")

In [47]:
%pwd

'C:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflow'

In [48]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Abdelm4lek/End-to-end-ML-project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="abdelm4lek"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5fd992211e7ef689322a2960f6357377e09a7cd5"

In [49]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [50]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_dirs, save_json

In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LightGBM
        schema =  self.schema.TARGET_COLUMN

        create_dirs([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Abdelm4lek/End-to-end-ML-project-with-MLflow.mlflow"
        )

        return model_evaluation_config


In [57]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [60]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_y = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_y)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="LGBMRegressor")
            else:
                mlflow.sklearn.log_model(model, "model")


In [61]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-11 23:43:59,056: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-11 23:43:59,058: INFO: common: yaml file: config\params.yaml loaded successfully]
[2025-06-11 23:43:59,060: INFO: common: yaml file: config\schema.yaml loaded successfully]
[2025-06-11 23:43:59,061: INFO: common: created directory at: artifacts]
[2025-06-11 23:43:59,062: INFO: common: created directory at: artifacts/model_evaluation]
🏃 View run bittersweet-worm-238 at: https://dagshub.com/Abdelm4lek/End-to-end-ML-project-with-MLflow.mlflow/#/experiments/0/runs/b9b402698f6d4cd7bd6e1ef45927e95f
🧪 View experiment at: https://dagshub.com/Abdelm4lek/End-to-end-ML-project-with-MLflow.mlflow/#/experiments/0


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: date: object